# Coleta de Dados do Sentinel-2 usando Google Earth Engine

Este notebook demonstra como coletar dados do Sentinel-2 para áreas de interesse definidas em arquivos KML.

In [1]:
import sys
sys.path.append('..')

import ee
from pathlib import Path
from src.data.gee_collector import SentinelCollector

## 1. Configurar o Ambiente

Primeiro, vamos configurar os diretórios e inicializar os componentes necessários:

In [2]:
# Configurar diretórios
data_dir = Path('../data')
kml_dir = data_dir / 'kmls'  # Diretório para todos os KMLs

# Criar diretório se não existir
kml_dir.mkdir(parents=True, exist_ok=True)

# Inicializar o coletor
collector = SentinelCollector(train_val_dir=str(kml_dir))

In [3]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'bands': ['B8', 'B4', 'B11'],  # Bandas relevantes para vegetação
    'folder': 'sentinel2_series'  # Pasta no Google Drive
}

In [4]:
# Processar cada KML independentemente
train_val_data = collector.dataset_manager.load_train_val_data()

Loaded 1 geometries from a8ec4540-3574-443d-99fb-107f4dcb42db.kml
Loaded 1 geometries from d2a4bf0d-45c7-4c17-b160-7dd440813d5c.kml
Loaded 1 geometries from b970c077-2956-47d7-98e3-9f4ee5cc5a54.kml
Loaded 1 geometries from c827e9cc-5b8a-4dfa-9c8a-630659550f1c.kml
Loaded 1 geometries from 99902fd9-66ed-463b-8e16-67f5f408bc94.kml
Loaded 1 geometries from 561564b9-a657-4b8e-bfee-585cf3400928.kml
Loaded 1 geometries from 2f48597d-7a88-4b9c-b464-f6ffd43db198.kml
Loaded 1 geometries from 4c64de73-22cd-4491-a2af-3139352c46a9.kml
Loaded 1 geometries from 2a98a92f-2e4d-469e-9fc4-5c165e52c870.kml
Loaded 1 geometries from 5e783152-5989-4959-8d3a-1a7d0507bf5a.kml
Loaded 1 geometries from 221495e5-d593-4b0f-b10e-85edaf5fab4e.kml
Loaded 1 geometries from ff45ecfe-5128-4db5-843c-f719686a4077.kml
Loaded 1 geometries from 339ea663-b9e5-4901-b5f4-84fb0650b55c.kml
Loaded 1 geometries from ca71104b-6515-421d-aa73-9d186e43c147.kml
Loaded 1 geometries from 386500de-a96a-41e2-aee0-ae1aac480927.kml
Loaded 1 g

In [ ]:
for geometries, kml_name, kml_path in train_val_data:
    print(f"\nProcessando KML: {kml_name}")
    print(f"Número de geometrias: {len(geometries)}")
    
    # Criar collection específica para este KML
    collection = collector.get_sentinel_collection(
        regions=geometries,
        years_back=2,  # Últimos 2 anos de dados
        max_cloud_cover=20  # Máximo de 20% de cobertura de nuvens
    )
    
    # Exportar imagens para este KML
    collector.export_to_drive(
        collection=collection,
        regions=geometries,
        output_prefix=f"sentinel2_{kml_name}",
        **export_params
    )
    
    print(f"Exportação iniciada para {kml_name}")


Processando KML: a8ec4540-3574-443d-99fb-107f4dcb42db
Número de geometrias: 1

Preparing to export 22 images for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db...
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-04-20_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-05-15_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-06-09_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-07-04_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-08-13_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-09-17_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-10-02_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-11-01_region_0
Started export task for sentinel2_a8ec4540-3574-443d-99fb-107f4dcb42db_2023-12-01_region_0
Started export ta

## 2. Carregar Regiões

Vamos carregar todas as regiões dos KMLs:

In [3]:
# Carregar todas as regiões dos KMLs
regions, region_names = geometry_handler.load_kml_directory(kml_dir)
print(f"Número total de regiões: {len(regions)}")

Loaded 1 geometries from a8ec4540-3574-443d-99fb-107f4dcb42db.kml
Loaded 1 geometries from d2a4bf0d-45c7-4c17-b160-7dd440813d5c.kml
Loaded 1 geometries from b970c077-2956-47d7-98e3-9f4ee5cc5a54.kml
Loaded 1 geometries from c827e9cc-5b8a-4dfa-9c8a-630659550f1c.kml
Loaded 1 geometries from 99902fd9-66ed-463b-8e16-67f5f408bc94.kml
Loaded 1 geometries from 561564b9-a657-4b8e-bfee-585cf3400928.kml
Loaded 1 geometries from 2f48597d-7a88-4b9c-b464-f6ffd43db198.kml
Loaded 1 geometries from 4c64de73-22cd-4491-a2af-3139352c46a9.kml
Loaded 1 geometries from 2a98a92f-2e4d-469e-9fc4-5c165e52c870.kml
Loaded 1 geometries from 5e783152-5989-4959-8d3a-1a7d0507bf5a.kml
Loaded 1 geometries from 221495e5-d593-4b0f-b10e-85edaf5fab4e.kml
Loaded 1 geometries from ff45ecfe-5128-4db5-843c-f719686a4077.kml
Loaded 1 geometries from 339ea663-b9e5-4901-b5f4-84fb0650b55c.kml
Loaded 1 geometries from ca71104b-6515-421d-aa73-9d186e43c147.kml
Loaded 1 geometries from 386500de-a96a-41e2-aee0-ae1aac480927.kml
Loaded 1 g

## 3. Coletar Imagens do Sentinel-2

Agora vamos coletar todas as imagens válidas dos últimos 5 anos:

In [4]:
# Coletar imagens
collection = collector.get_sentinel_collection(
    regions=regions,
    years_back=2,  # Últimos 5 anos de dados
    max_cloud_cover=20  # Máximo de 20% de cobertura de nuvens
)

In [7]:
regions

[ee.Geometry({
   "functionInvocationValue": {
     "functionName": "GeometryConstructors.Polygon",
     "arguments": {
       "coordinates": {
         "constantValue": [
           [
             [
               -54.15978194,
               -28.17027861
             ],
             [
               -54.1598075,
               -28.17096056
             ],
             [
               -54.15981944,
               -28.17129972
             ],
             [
               -54.15988,
               -28.17231194
             ],
             [
               -54.15991806,
               -28.17319722
             ],
             [
               -54.15989861,
               -28.17367389
             ],
             [
               -54.15992361,
               -28.174445
             ],
             [
               -54.159955,
               -28.17569361
             ],
             [
               -54.16000111,
               -28.17763194
             ],
             [
               -

## 4. Exportar para Google Drive

Vamos exportar todas as imagens válidas para o Google Drive:

In [5]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'bands': ['B8', 'B4', 'B11'],  # Bandas relevantes para vegetação
    'folder': 'sentinel2_series',  # Pasta no Google Drive
    'region_names': region_names  # Nomes das regiões dos KMLs
}

# Exportar todas as imagens
collector.export_to_drive(
    collection=collection,
    regions=regions,
    output_prefix='sentinel2',
    **export_params
)


Preparing to export 17233 images for each of 633 regions...

Processing region: a8ec4540-3574-443d-99fb-107f4dcb42db (1/633)
Error exporting image 0 for region a8ec4540-3574-443d-99fb-107f4dcb42db: User memory limit exceeded.


KeyboardInterrupt: 

## 5. Monitoramento e Próximos Passos

### Monitoramento
As tarefas de exportação serão executadas no Google Earth Engine. Você pode monitorar o progresso em:
https://code.earthengine.google.com/tasks

### Organização dos KMLs
- Coloque todos os seus arquivos KML no diretório `kmls`
- O script irá processar todos os KMLs encontrados no diretório
- Você verá um resumo de quantas geometrias foram carregadas de cada arquivo

### Após a Exportação
1. As imagens serão exportadas para a pasta 'sentinel2_series' no seu Google Drive
2. Cada imagem terá o nome no formato: tiff/sentinel2_YYYY-MM-DD
3. As imagens serão exportadas no formato Cloud Optimized GeoTIFF

### Importante
Certifique-se de que você tem:
- Acesso configurado ao Google Earth Engine
- Espaço suficiente no Google Drive
- KMLs bem definidos para suas áreas de interesse